In [1]:
import pandas as pd
import string

# Verisetinin Okunması

In [2]:
halk_dataset = pd.read_csv("../dataset/halk.csv", usecols=["sarki_sozu"])
sanat_dataset = pd.read_csv("../dataset/sanat.csv", usecols=["sarki_sozu"])
ilahi_dataset = pd.read_csv("../dataset/ilahi.csv", usecols=["sarki_sozu"])
pop_dataset = pd.read_csv("../dataset/pop.csv", usecols=["sarki_sozu"])
rock_dataset = pd.read_csv("../dataset/rock.csv", usecols=["sarki_sozu"])
rap_dataset = pd.read_csv("../dataset/rap.csv", usecols=["sarki_sozu"])

# Veri Etiketleme
halk_dataset["sarki_turu"] = [0 for _ in range(200)]
sanat_dataset["sarki_turu"] = [1 for _ in range(200)]
ilahi_dataset["sarki_turu"] = [2 for _ in range(200)]
pop_dataset["sarki_turu"] = [3 for _ in range(200)]
rock_dataset["sarki_turu"] = [4 for _ in range(200)]
rap_dataset["sarki_turu"] = [5 for _ in range(200)]

df_dataset = pd.concat([halk_dataset, sanat_dataset, ilahi_dataset, pop_dataset, rock_dataset, rap_dataset], axis=0, ignore_index=True)

df_dataset = df_dataset.reset_index(drop=True)

# Verilerin Temizlenmesi

In [3]:
# Getting stop_words
my_file = open("../raw_data/stop_words.txt", "r")
stop_words = my_file.read()
stop_words = stop_words.replace('\n', ' ').split(" ")
my_file.close()

def clean_text(text):
    # Satır boşluklarının kaldırılması
    text = text.replace("\n\n", "\n")

    # Satır Sonlarındaki Boşlukların Kaldırılması
    lines = text.split("\n")
    temp_text = ""
    for line in lines:
        temp_text += line.rstrip() + "\n"
    text = temp_text

    # Noktalama İşaretlerinin Kaldırılması
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Küçük Harf Dönüşümü
    text = text.lower()

    # stop word'lerin kaldırılması
    lines = text.split('\n')
    for i, line in enumerate(lines):
        words = line.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        lines[i] = ' '.join(filtered_words)
    text = '\n'.join(lines)
    
    return text



df_dataset["sarki_sozu"] = [clean_text(lyrics) for lyrics in df_dataset.loc[:,"sarki_sozu"]]

##### [CLS] ve [SEP] İşaretlerinin Eklenmesi

In [4]:
df_dataset["sarki_sozu"] = ["[CLS]" + lyrics.replace("\n", "[SEP]") for lyrics in df_dataset.loc[:,"sarki_sozu"]]

### Kelimeleri Ayırma

In [5]:
split_chars = ["[CLS]", " ", "[SEP]"]

splitted_words_for_all_lyrics_in_data = []
for lyrics in df_dataset.loc[:, "sarki_sozu"]:
    words = []
    for char in split_chars:
        lyrics = lyrics.replace(char, " ")
    words = lyrics.split()
    splitted_words_for_all_lyrics_in_data.append(words)
kelimeler = splitted_words_for_all_lyrics_in_data

In [6]:
max_length = 0
for words in kelimeler:
    if len(words) > max_length:
        max_length = len(words)

print(max_length)

822


In [11]:
#verileri kullanabilirsin

In [ ]:
import pos_tagger

for data in dataset:
    pos_tags_list = []
    verb_rate_list = []
    for lyrics in data.loc[:, "sarki_sozu"]:
        pos_tags = pos_tagger.get_pos_tags(lyrics.replace("[CLS]", "").replace("[SEP]", "\n"))
        pos_tags_list.append(pos_tags)
        verb_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Verb.name) / len(pos_tags), 2))
    data["pos_tagler"] = pos_tags_list
    data["fiil_orani"] = verb_rate_list


In [ ]:
for data in dataset:
    print(sum(data.loc[:, "fiil_orani"]) / len(data.loc[:, "fiil_orani"]))